# Create Dashboard on DASH 

In [2]:
# Import Libraries

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from sklearn import datasets, linear_model
from sklearn.metrics import r2_score
import dash
from dash import dcc, html
import pandas as pd
import dash
from dash import dcc, html


In [ ]:
#Import all dfs for our dashbaord

In [3]:
df1 = pd.read_csv('dash_df1.csv')
df2 = pd.read_csv('dash_df2.csv')
df3 = pd.read_csv('dash_df3.csv')
df4 = pd.read_csv('dash_df4.csv')
df5 = pd.read_csv('dash_df5.csv')
df6 = pd.read_csv('dash_df6.csv')
df7 = pd.read_csv('dash_df2.csv')

In [4]:
# Essential to 6th Visualisation

df6 = pd.read_csv('dfplotf.csv')
import string
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import statistics

# Download the required NLTK resources
nltk.download('punkt')
nltk.download('vader_lexicon')

# Create SentimentIntensityAnalyzer instance
sia = SentimentIntensityAnalyzer()


# Function to calculate sentiment scores
def calculate_sentiment_scores(comment):
    sia = SentimentIntensityAnalyzer()
    sentiment_score = sia.polarity_scores(comment)['compound']
    return sentiment_score

# Function to calculate word frequencies
def process_comments_and_get_metrics(df6):
    comments = ' '.join(df6['Comment']).lower()
    comments = comments.translate(str.maketrans('', '', string.punctuation))  
    word_list = comments.split()

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    word_list = [word for word in word_list if word not in stop_words]

    # Calculate sentiment scores for each comment
    df6['Sentiment'] = df6['Comment'].apply(calculate_sentiment_scores)
    
    # Calculate word frequencies
    word_freq = Counter(word_list)
    
    # Ensure the lengths match
    common_length = min(len(word_freq), len(df6))
    
    return pd.DataFrame({
        'Word': list(word_freq.keys())[:common_length],
        'Frequency': list(word_freq.values())[:common_length],
        'Sentiment': df6['Sentiment'].iloc[:common_length].tolist()  
    })

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aad.sray\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\aad.sray\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [5]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
from wordcloud import WordCloud
import io
import base64
import matplotlib.pyplot as plt
import string
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
nltk.download("vader_lexicon")
nltk.download('punkt')
nltk.download('stopwords')
import numpy as np
import pandas as pd
import statistics
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import dash
from dash import dcc, html
import plotly.express as px
from nltk.corpus import stopwords
from collections import Counter
import string


# Import Libraries

df1 = pd.read_csv('datavisnew.csv')
df1['Time'] = pd.to_datetime(df1['Time'])
df2 = pd.read_csv('datavismins.csv')
df2['Time'] = pd.to_datetime(df2['Time'])
df3 = pd.read_csv('visdf.csv')
df3['Time'] = pd.to_datetime(df3['Time'])
df3['Hour'] = df3['Time'].dt.hour
df4 = pd.read_csv('dfplotd.csv')
df5 = pd.read_csv('plotedf.csv')
df_pie = df5.groupby('Sentiment Category').size().reset_index(name='Count')
df6 = pd.read_csv('dfplotf.csv')
df7 = pd.read_csv('dash_df2.csv')
df7['Time'] = pd.to_datetime(df7['Time'])
df7['Hour'] = df7['Time'].dt.hour

# Heat map code pre-reqs 

heatmap_data = df7.pivot_table(values='Score', index='Hour', aggfunc='mean').reset_index()
max_columns = 6
num_rows = (len(heatmap_data) + max_columns - 1) // max_columns
heatmap_matrix = np.zeros((num_rows, max_columns))
indices = heatmap_data['Hour'].values
heatmap_matrix[np.arange(len(indices)) // max_columns, np.arange(len(indices)) % max_columns] = heatmap_data['Score']

# Generate word cloud topic
def generate_wordcloud(topic):
    comments = ' '.join(df4[df4['Topic'] == topic]['Comment'])
    if not comments:
        return None

    # Set style parameters 
    wordcloud = WordCloud(width=800, height=400, background_color='black', colormap='plasma', collocations=False).generate(comments)

    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    img_buf = io.BytesIO()
    plt.savefig(img_buf, format='png')
    img_buf.seek(0)
    img_base64 = base64.b64encode(img_buf.read()).decode('utf-8')
    plt.close()
    return f'data:image/png;base64,{img_base64}'

# Get df metrics
df_metrics = process_comments_and_get_metrics(df6)

# Create Dash App
app = dash.Dash(__name__)

# Slider mark labels (hourly)
marks = {
    i: {'label': pd.to_datetime(i, unit='s').strftime('%H') + 'hr', 'style': {'transform': 'rotate(-90deg)', 'margin-left': '-10px'}}
    for i in range(int(df1['Time'].min().timestamp()), int(df1['Time'].max().timestamp()) + 1, 3600)
}


# Layout App
app.layout = html.Div(children=[
    html.H1("Dashboard: Comment Analysis"),

    # First Graph: Pie chart
    dcc.Graph(
        id='sentiment-pie-chart',
        figure=px.pie(
            df_pie,
            values='Count',
            names='Sentiment Category',
            title='Sentiment Category Distribution',
            color_discrete_sequence=px.colors.sequential.Plasma,
        ).update_layout(
            xaxis=dict(title_text='Sentiment Category'),
            yaxis=dict(title_text='Count'),
        )
    ),

    # Second Graph: Histogram
    dcc.Graph(
        id='hourly-histogram',
        figure=px.histogram(
            df3,
            x='Hour',
            title='Distribution of Comments Over Hours',
            color_discrete_sequence=['#7201a8']
        ).update_layout(
            xaxis_title='Hour',
            yaxis_title='Comment Count'
        )
    ),

    # Third Graph: Time Series Plot
    html.Div([
        html.H1("Comment Scores Over Time"),

        # Add sliders
        dcc.RangeSlider(
            id='time-slider',
            marks=marks,
            min=df1['Time'].min().timestamp(),
            max=df1['Time'].max().timestamp(),
            step=3600,  
            value=[df1['Time'].min().timestamp(), df1['Time'].max().timestamp()]
        ),

        dcc.Graph(
            id='line-chart',
            config={'displayModeBar': False}  
        )
    ]),

    # Fourth Graph: Comment Scores Bar Plot
    html.Div([
        html.H1("Comment Scores Bar Plot"),

        # Add dropdown with 4 features we want to display (ID, Score, Hour, Minute)
        dcc.Dropdown(
            id='x-axis-dropdown',
            options=[
                {'label': column, 'value': column}
                for column in ['ID', 'Score', 'Hour', 'Minutes']  
            ],
            value='ID',  
            style={'width': '50%'}
        ),

        dcc.Graph(
            id='bar-chart',
            config={'displayModeBar': False}  
        )
    ]),

    # Fifth Element: Topic Word Cloud Generator
    html.Div([
        html.H1("Topic Word Cloud Generator"),
        dcc.Dropdown(
            id='topic-dropdown',
            options=[{'label': f'Topic {i}', 'value': i} for i in range(1, 11)],  
            value=1,  
            multi=False
        ),
        html.Div(id='wordcloud-container')
    ]),
    
    # Sixth Element Word Frequency and Sentiment Analysis
    html.Div([
        html.H1("Word Sentiment Analysis - Sentiment and Frequency"),

        # Plotly Graph 
        dcc.Graph(
            id='scatter-plot',
            figure=px.scatter(
                df_metrics,
                x='Frequency',
                y='Sentiment',
                text='Word',
                title='Sentiment and Frequency Scatter Plot',
                color='Frequency',  # Set color based on Frequency
                color_continuous_scale='plasma',  # Set color scale to 'plasma'
            ).update_traces(mode='markers', marker=dict(size=10))
        )
    ]),
    
    # Seventh Element: Heatmap
    html.Div([
        html.H1("Heatmap Graph"),
    
        dcc.Graph(
            id='heatmap-graph',
            figure=px.imshow(
                heatmap_matrix,
                labels=dict(x="Hour", y="Score"),
                x=list(range(max_columns)),
                y=list(range(num_rows)),
                color_continuous_scale='Plasma'
            ).update_layout(
                title="Average Score by Hour (4x6 Grid)",
                xaxis_title="Column",
                yaxis_title="Hour (Row)",
                coloraxis_colorbar=dict(yanchor="middle", y=0.5, x=-0.15),
            )
        )
    ]),
])


# Define callback for Time Series Plot
@app.callback(
    Output('line-chart', 'figure'),
    [Input('time-slider', 'value')]
)
def update_chart(selected_time_range):
    start_time, end_time = selected_time_range
    filtered_df = df1[(df1['Time'] >= pd.to_datetime(start_time, unit='s')) & (df1['Time'] <= pd.to_datetime(end_time, unit='s'))]

    # Add color scale using 'Score'
    fig = px.scatter(filtered_df, x='Time', y='Score', color='Score',title='Score Over Time (Zoomed In)', color_continuous_scale='plasma', opacity=0.6)
                     
    # Update color bar position to the right
    fig.update_layout(coloraxis_colorbar=dict(yanchor="middle", y=0.5, x=-0.25))

    return fig

# Define callback for Comment Scores Bar Plot
@app.callback(
    Output('bar-chart', 'figure'),
    [Input('x-axis-dropdown', 'value')]
)
def update_bar_chart(selected_column):
    # Create 'Score' bar chart
    fig = px.bar(df2, x=selected_column, y='Score', title=f'Comment Scores - {selected_column}', color='Score',
                 color_continuous_scale='Plasma')

    fig.update_layout(coloraxis_colorbar=dict(yanchor="middle", y=0.5, x=-0.25))

    return fig

# Define callback for Topic Word Cloud Generator
@app.callback(
    Output('wordcloud-container', 'children'),
    [Input('topic-dropdown', 'value')]
)
def update_wordcloud(selected_topic):
    wordcloud_img = generate_wordcloud(selected_topic)
    if wordcloud_img:
        return html.Img(src=wordcloud_img, style={'width': '80%', 'height': '80%'})
    else:
        return html.Div("No comments available for the selected topic.")

# Run app
if __name__ == '__main__':
    app.run_server(debug=True, port=8086)


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\aad.sray\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aad.sray\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aad.sray\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
